In [195]:
'''
Code intended to evaluate the performances of Orbslam3 and DSO using custom dataset in TUM format.
Requirements.
1) Container containing DSO (custom)
2) Container containing ORBSLAM3 (custom)
3) Folder containing the datasets
4) Evo tools ;D
'''
import docker
import os
import sys
import tarfile
import numpy as np
import pandas as pd
import yaml
import logging
from tqdm import tqdm

In [ ]:
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [196]:
# Load config
root_path = os.getcwd()
config_path = os.path.join(root_path, "config")
docker_config = os.path.join(config_path, "docker_containers.yaml")
local_config = os.path.join(config_path, "local_host_params.yaml")
print("Current config path: {}".format(config_path))
try:
    with open(docker_config, "r") as yamlfile:
        docker_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t -Docker config loaded;")
except OSError:
    print("Could not open/read file:", docker_config)
    sys.exit()

try:
    with open(local_config, "r") as yamlfile:
        local_data_cfg = yaml.load(yamlfile, Loader=yaml.FullLoader)
        print("\t -Local config loaded;")
except OSError:
    print("Could not open/read file:", local_config)
    sys.exit()

Current config path: /home/croc/Documenti/Repositories/evoTools/notebooks/config
	 -Docker config loaded;
	 -Local config loaded;


In [197]:
######################################################################
# DOCKER SET-UP
######################################################################
# Get the configurations for the containers
dso_container_config = docker_data_cfg["Containers"][0]["DSO"]
orb_container_config = docker_data_cfg["Containers"][1]["ORBSLAM3"]

# Get the client
client_dk = docker.from_env()

# Retrieve the containers
orb_container = [cont for cont in client_dk.containers.list(all=True) if cont.short_id == orb_container_config["Id"]][0]
dso_container = [cont for cont in client_dk.containers.list(all=True) if cont.short_id == dso_container_config["Id"]][0]

# Start the two containers
if orb_container.status == 'exited':
    orb_container.start()
    print("Container {} Started".format(orb_container.name))
elif orb_container.status =="running":
     print("Container {} Already started".format(orb_container.name))

if dso_container.status == 'exited':
    dso_container.start()
    print("Container {} Started".format(dso_container.name))
elif dso_container.status =="running":
    print("Container {} Already started".format(dso_container.name))


Container test_orbslam3 Already started
Container test_dso Already started


In [198]:
######################################################################
# DATASET LOADING
######################################################################

#Common Parameters
datasets_root_path = local_data_cfg["Datasets"]["root_path"]
datasets_map_folder = local_data_cfg["Datasets"]["map_folder"]
datasets_data_folder = local_data_cfg["Datasets"]["data_folder"]
datasets_eva_folder = local_data_cfg["Datasets"]["out_eva_folder"]

#Datasets selected
dataset_list = local_data_cfg["Datasets"]["Scenarios"]

In [203]:
# Sub-Datasets Discovering
dataset_selected = list(dataset_list.keys())[0]
folder_dataset_sel=dataset_list[dataset_selected]["folder_name"]
path_dataset = os.path.join(datasets_root_path, folder_dataset_sel)
if os.path.isdir(path_dataset):
    elem = [elem for elem in os.listdir(os.path.join(path_dataset, datasets_data_folder))]
    print("Discovering subsets of {}: {}".format(dataset_selected,elem))
else:
    elem = []

Discovering subsets of Corridor_A: ['Corridor_A_D_190', 'Corridor_A_D_60', 'Corridor_A_D_255', 'Corridor_A_D_85', 'Corridor_A_D_127', 'Corridor_A_L', 'Corridor_A_D_25']


In [206]:
# We'll try to process just the CorridorA
curr_dataset = elem[5]
path_sub_dataset = os.path.join(path_dataset, curr_dataset)
print("Processing '{}' ...".format(path_sub_dataset))

print("Compressing")


Processing '/home/croc/Documenti/Trajectories/excasi/Corridor_A_L' ...
Compressing
